
# Data cleaning  for modeling

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [138]:
#load dataset
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
print(train.shape)
print(test.shape)

(1460, 81)
(1459, 80)


In [71]:
combined=pd.concat([train,test])
combined.shape #2919
#combined.iloc[:1460:,:] #train
#combined.iloc[1460:,:] #test

C:\Users\samuelmao\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(2919, 81)

In [72]:
combined=combined.reset_index()

In [73]:
combined.iloc[1458:1464,:]

,index,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,...,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
1458,1458,1078,0,0,NaN,2,1Fam,TA,Mn,49.0,...,WD,0,Pave,5,1078.0,AllPub,366,1950,1996,2010
1459,1459,1256,0,0,NaN,3,1Fam,TA,No,830.0,...,WD,0,Pave,6,1256.0,AllPub,736,1965,1965,2008
1460,0,896,0,0,NaN,2,1Fam,TA,No,468.0,...,WD,120,Pave,5,882.0,AllPub,140,1961,1961,2010
1461,1,1329,0,0,NaN,3,1Fam,TA,No,923.0,...,WD,0,Pave,6,1329.0,AllPub,393,1958,1958,2010
1462,2,928,701,0,NaN,3,1Fam,TA,No,791.0,...,WD,0,Pave,6,928.0,AllPub,212,1997,1998,2010
1463,3,926,678,0,NaN,3,1Fam,TA,No,602.0,...,WD,0,Pave,7,926.0,AllPub,360,1998,1998,2010


In [74]:
colnames=combined.columns.tolist()

In [75]:
x_vars=colnames[:-1]
y_vars=colnames[-1]

### Variables that need to be dummified and combined

 Need to encode some fields based on their definition and drop the original.
 
 Exterior1st, Exterior2nd (Exterior covering on house)


BsmtFinType1, BsmtFinType2, BsmtFinSF1 (Type 1 finished square feet), BsmtFinSF2 (Type 1 finished square feet)


BsmtFullBath, BsmtHalfBath  (number of type of bathroom in the basement)


OpenPorchSF ( Open porch area in square feet)
EnclosedPorch (Enclosed porch area in square feet)
3SsnPorch (Three season porch area in square feet)
ScreenPorch (Screen porch area in square feet)


MasVnrType, MasVnrArea


PoolQC, PoolArea

In [76]:
combined.dtypes

index              int64
1stFlrSF           int64
2ndFlrSF           int64
3SsnPorch          int64
Alley             object
BedroomAbvGr       int64
BldgType          object
BsmtCond          object
BsmtExposure      object
BsmtFinSF1       float64
BsmtFinSF2       float64
BsmtFinType1      object
BsmtFinType2      object
BsmtFullBath     float64
BsmtHalfBath     float64
BsmtQual          object
BsmtUnfSF        float64
CentralAir        object
Condition1        object
Condition2        object
Electrical        object
EnclosedPorch      int64
ExterCond         object
ExterQual         object
Exterior1st       object
Exterior2nd       object
Fence             object
FireplaceQu       object
Fireplaces         int64
Foundation        object
                  ...   
LotShape          object
LowQualFinSF       int64
MSSubClass         int64
MSZoning          object
MasVnrArea       float64
MasVnrType        object
MiscFeature       object
MiscVal            int64
MoSold             int64


### houses with different exterior: one-hot encoding

In [77]:
part=pd.get_dummies(combined[["Exterior1st","Exterior2nd"]],drop_first=True,dummy_na=True) #dummify both the exterior column
part.columns=part.columns.str.replace("1st","") #change name of the column
part.columns=part.columns.str.replace("2nd","")
part=part.groupby(part.columns, axis=1).sum() #group by the same column and thus summing duplicate as two
part=part.astype(float)
part[part.groupby(part.columns, axis=1).sum()>1]=part[part.groupby(part.columns, axis=1).sum()>1]-1# list comprehension?

In [78]:
#drop original dataset exterior and then merge
combined=combined.drop(["Exterior1st"],axis=1)
combined=combined.drop(["Exterior2nd"],axis=1)

In [79]:
print(combined.shape)
print(part.shape)

(2919, 80)
(2919, 19)


In [80]:
combined=combined.join(part)
combined.shape#98

(2919, 99)

### Basement Dummyfication
BsmtFinType1, BsmtFinType2, BsmtFinSF1 (Type 1 finished square feet), BsmtFinSF2 (Type 1 finished square feet) BsmtUnfSF: Unfinished square feet of basement area TotalBsmtSF: Total square feet of basement area

In [81]:
#new=train[["BsmtFinType1","BsmtFinSF1" , "BsmtFinType2", "BsmtFinSF2"]]
new=combined[["BsmtFinType1","BsmtFinSF1" ,"BsmtFinType2", "BsmtFinSF2" ]]
new.head(4)

,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2
0,GLQ,706.0,Unf,0.0
1,ALQ,978.0,Unf,0.0
2,GLQ,486.0,Unf,0.0
3,ALQ,216.0,Unf,0.0


In [82]:
tmp=pd.get_dummies(new["BsmtFinType1"],drop_first=True,dummy_na=True)
tmp.head(3)

,BLQ,GLQ,LwQ,Rec,Unf,nan
0,0,1,0,0,0,0
1,0,0,0,0,0,0
2,0,1,0,0,0,0


In [83]:
#use pivot like spread in R: add square fit to the value and spread the value of each type into different columns
tmp=new.pivot(columns="BsmtFinType1", values="BsmtFinSF1").fillna(0)
tmp
tmp2=new.pivot(columns="BsmtFinType2", values="BsmtFinSF2").fillna(0)
tmp2
tmp_fin=tmp+tmp2
tmp_fin.shape# 1460

(2919, 7)

In [84]:
combined=combined.drop(columns=["BsmtFinType1","BsmtFinSF1" ,"BsmtFinType2", "BsmtFinSF2" ],axis=1)

In [85]:
combined=combined.join(tmp_fin) #join dummified bsmt to training dataset

In [86]:
combined = combined.rename(columns={np.nan: 'Bsmt_nan'}) #rename nan dataset to basemet nan

In [87]:
#rename using dictionary
combined = combined.rename(columns=dict(zip(('ALQ', 'BLQ', 'GLQ', 'LwQ', 'Rec', 'Unf'),('Bsmt_ALQ', 'Bsmt_BLQ', 'Bsmt_GLQ', 'Bsmt_LwQ', 'Bsmt_Rec', 'Bsmt_Unf'))))

### Bathroom dummification (combining one and half together)
BsmtFullBath, BsmtHalfBath  (number of type of bathroom in the basement)


In [88]:
#pd.options.display.max_rows=20
#train.dtypes
combined[["BsmtFullBath","BsmtHalfBath"]]
combined["BsmtHalfBath"]=combined["BsmtHalfBath"]*0.5

In [89]:
combined["BsmtBath"]=combined["BsmtFullBath"]+combined["BsmtHalfBath"]
combined=combined.drop(columns=["BsmtFullBath","BsmtHalfBath"],axis=1)

### Porch
Add all the porch into sqft

OpenPorchSF ( Open porch area in square feet) EnclosedPorch (Enclosed porch area in square feet) 3SsnPorch (Three season porch area in square feet) ScreenPorch (Screen porch area in square feet)

In [90]:
combined["TotalPorchSF"]=combined["OpenPorchSF"]+combined["EnclosedPorch"]+combined["3SsnPorch"]+combined["ScreenPorch"]
combined=combined.drop(columns=["OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch"],axis=1)

### Masonry veneer area

In [91]:
#Masonry veneer area
combined[["MasVnrType", "MasVnrArea"]]
tmp=combined.pivot(columns="MasVnrType", values="MasVnrArea").fillna(0)
#sum(tmp[np.nan])
tmp

MasVnrType,nan,BrkCmn,BrkFace,None,Stone
0,0.0,0.0,196.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,162.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,350.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,186.0
7,0.0,0.0,0.0,0.0,240.0
8,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0


In [92]:
tmp =tmp.rename(columns=dict(zip(('BrkCmn', 'BrkFace', 'None', 'Stone'),('MasBrkCmn', 'MasBrkFace', 'MasNone', 'MasStone'))))
tmp=tmp.drop(columns=np.nan)

In [93]:
combined=combined.join(tmp) #join dummified bsmt to training dataset

### Ordinal label encoding:
 ['ExterQual', 'ExterCond','BsmtCond','HeatingQC', 'KitchenQual', 
           'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']


In [139]:
tmp_cols = ['ExterQual', 'ExterCond','BsmtCond','HeatingQC', 'KitchenQual', 
           'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC',"BsmtQual"]
tmp_dic = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa':2, 'Po':1}

for c in tmp_cols:
    combined[c]=combined[c].map(lambda x: tmp_dic.get(x, 0))

    

[4,
 4,
 4,
 3,
 4,
 4,
 5,
 4,
 3,
 3,
 3,
 5,
 3,
 4,
 3,
 3,
 3,
 0,
 3,
 3,
 5,
 3,
 4,
 4,
 3,
 4,
 3,
 5,
 3,
 3,
 3,
 3,
 5,
 3,
 5,
 5,
 4,
 3,
 3,
 0,
 3,
 3,
 4,
 4,
 3,
 5,
 5,
 4,
 3,
 3,
 4,
 3,
 4,
 5,
 3,
 3,
 4,
 4,
 5,
 3,
 4,
 3,
 4,
 3,
 4,
 5,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 2,
 4,
 3,
 3,
 3,
 3,
 4,
 5,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 0,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 4,
 0,
 4,
 3,
 5,
 2,
 3,
 3,
 4,
 3,
 4,
 5,
 4,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 3,
 3,
 3,
 4,
 3,
 5,
 3,
 3,
 3,
 3,
 4,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 3,
 4,
 4,
 4,
 3,
 5,
 3,
 4,
 3,
 3,
 0,
 5,
 4,
 4,
 4,
 5,
 4,
 3,
 3,
 3,
 3,
 5,
 4,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 4,
 3,
 5,
 3,
 4,
 2,
 0,
 4,
 2,
 3,
 4,
 2,
 4,
 5,
 4,
 3,
 4,
 4,
 3,
 4,
 5,
 3,
 3,
 5,
 4,
 3,
 2,
 4,
 3,
 4,
 3,
 3,
 4,
 4,
 3,
 4,
 4,
 4,
 4,
 3,
 4,
 3,
 3,
 4,
 4,
 4,
 4,
 3,
 5,
 3,
 4,
 3,
 3,
 4,
 3,
 4,
 3,
 3,
 4,
 3,
 4,
 4,
 5,
 3,
 4,
 3,
 3,
 4,
 4,
 4,
 2,
 3,
 4,
 4,


In [95]:
combined.columns[combined.dtypes=="object"]

Index(['Alley', 'BldgType', 'BsmtExposure', 'CentralAir', 'Condition1',
       'Condition2', 'Electrical', 'Fence', 'Foundation', 'Functional',
       'GarageFinish', 'GarageType', 'Heating', 'HouseStyle', 'LandContour',
       'LandSlope', 'LotConfig', 'LotShape', 'MSZoning', 'MasVnrType',
       'MiscFeature', 'Neighborhood', 'PavedDrive', 'RoofMatl', 'RoofStyle',
       'SaleCondition', 'SaleType', 'Street', 'Utilities'],
      dtype='object')

In [96]:
tmp=combined.copy() #backup
tmp.shape #101

(2919, 102)

In [97]:
tmp1=pd.get_dummies(tmp[['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType',
       'Foundation', 'BsmtExposure', 'Heating', 'CentralAir',
       'Electrical', 'Functional', 'GarageType', 'GarageFinish', 'PavedDrive',
       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']], drop_first=True, dummy_na=True)

tmp=tmp.drop(columns=['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType',
       'Foundation', 'BsmtExposure', 'Heating', 'CentralAir',
       'Electrical', 'Functional', 'GarageType', 'GarageFinish', 'PavedDrive',
       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'])
tmp=tmp.join(tmp1)
tmp.shape

(2919, 238)

In [98]:
combined=tmp

In [99]:
combined.shape

(2919, 238)

### Encode MSSubClass(add something to the number) 

In [100]:
tmp=combined.copy()

In [101]:
tmp1=pd.get_dummies(tmp["MSSubClass"], prefix="MSSub" ,drop_first=True, dummy_na=True)
tmp=tmp.drop(columns="MSSubClass")
tmp=tmp.join(tmp1)
tmp.shape

(2919, 253)

In [102]:
combined=tmp

### Drop ID and index!

In [103]:
combined=combined.drop(columns="Id")

In [106]:
combined=combined.drop(columns="index")

In [107]:
combined.shape #1460 250

(2919, 251)

In [108]:
combined.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtCond,BsmtQual,BsmtUnfSF,ExterCond,ExterQual,FireplaceQu,Fireplaces,...,MSSub_75.0,MSSub_80.0,MSSub_85.0,MSSub_90.0,MSSub_120.0,MSSub_150.0,MSSub_160.0,MSSub_180.0,MSSub_190.0,MSSub_nan
0,856,854,3,3,4,150.0,3,4,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1262,0,3,3,4,284.0,3,3,3,1,...,0,0,0,0,0,0,0,0,0,0
2,920,866,3,3,4,434.0,3,4,3,1,...,0,0,0,0,0,0,0,0,0,0


### Check missing values

In [157]:
combined.columns[combined.isna().any()]

Index(['BsmtUnfSF', 'GarageArea', 'GarageCars', 'SalePrice', 'TotalBsmtSF',
       'BsmtBath'],
      dtype='object')

In [158]:
combined[['BsmtUnfSF', 'GarageArea', 'GarageCars', 'SalePrice', 'TotalBsmtSF',
       'BsmtBath']]=combined[['BsmtUnfSF', 'GarageArea', 'GarageCars', 'SalePrice', 'TotalBsmtSF',
       'BsmtBath']].fillna(0)

## Split train and test back

In [159]:
train=combined.iloc[:1460:,:] #train
test=combined.iloc[1460:,:] #test

In [160]:
test=test.drop(columns="SalePrice")

In [162]:
print(train.shape)
print(test.shape)

(1460, 251)
(1459, 250)


## Export cleaned data

In [163]:
train.to_csv("cleaned_train.csv",index=False)
test.to_csv("cleaned_test.csv",index=False)

### Feature engineering 2nd round: box-cox transformation